# Macro Nutrient Calculator 

# Purpose 
The purpose of this notebook is to create a macronutrient calulator. I will research variuos ways/ techniques to calculate macros and use what I've learnt to devise my own calculator.

In [22]:
import requests
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
import datetime
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import xml.etree.ElementTree as ET
from urllib.request import Request, urlopen
import json
from pandas.io.json import json_normalize
import re
import nbformat 

# Using prevoius notebooks 
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
%run 200-Acess_Database.ipynb

# Note
Any possible invalid inputs values will have to be address in the application development as the options will most likely be drop down options except for scale facto, height etc. 

# Calorie Calculator Research 

** BMR ** <br>
Formulas used to calculate the (BMR) basal metabloic rate or calories burnt while resting not including any actvity/ exercise during the day. All formula take into account gender, age, weight. Actvity level/ stress level is used to then calculate total calories required. <br>
1) Harris Benedict Formula <br>
Study Link : https://www.ncbi.nlm.nih.gov/pubmed/16576330 <bt>
This formula is used for calculating the (BMR) or basal/resting metabloic rate of a person. It is the most widely used formula for getting the BMR of a person. Thought to have caluclated pecision of +-14%. The stress/ activity scale can range from 1.2 -> 1.9 under this formula. <br>
2) Revised Harris Benedict Formula in 1984 <br> 
Study Link : https://academic.oup.com/ajcn/article-abstract/40/1/168/4691315?redirectedFrom=fulltext <br>
The orginal formula was revised in 1984 and is now thought to be more accurate for caluclating the (BMR) of obese patients then it was prevoiusly. The stress/ activity scale can range from 1.2 -> 1.9 under this formula. <br> 
3) The Mifflin St JEOR equation <br>
Study Link : https://www.ncbi.nlm.nih.gov/pubmed/15883556 <br>
Another (BMR) equation dervived in 1990. Thought to have a (BMR) calculation percision of +-10%. The stress/ activity scale can range from 1.2 -> 1.9 under this formula. <br>
4) The Schofield Equation <br>
Study Link: https://www.ncbi.nlm.nih.gov/pubmed/4044297 <br>
The Schofield Equation takes into consideration gender, age, and weight. Height is not required.  The final result is given as a range and is adjusted upward or downward from the calculated value based on the 'standard error of estimated' or SEE (provided by the author).  Individuals that have a much higher lean body mass (reduced body fat), should use the upper end of the range, while individuals with a greater body fat percentage should use the lower end of the range. The stress/ actvitiy level is on a broader scale with more levels of actvity to choose from when compared with the prevoius equations. <br>
5) Institute of Medicine - Estimated Energy Requirement (EER) <br>
Study reference : https://www.nal.usda.gov/sites/default/files/fnic_uploads/energy_full_report.pdf <br>
This formula is used for caluclating (EER) or estimated energy requirement. Which means the caloires required after taking into consideration (BMR) and activity level. <br>

** TEF ** <br>
The thermic effect of food is the energy required for digestion, absorption, and disposal of ingested nutrients. Accepted avergaes include: Carbohydrates: 5 to 15% of the energy consumed, Protein: 20 to 35% and Fats: at most 5 to 15%. In order to accuratly predict the (BMR) of a person we must subtract this number from our intial (BMR) estimates. <bt>
Study Link: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4816895/ <br>

# My formula
** BMR **  <br>
I'm going to use the average of the first four formuals to get my estimated (BMR). <br>
To Get the EER I'll take into account the following factors
** TEF ** <br>
To get my number for TEF I'll get the halfway the average of the median of the three expected estimates of TEF for the three maronutrietns   Protein, Carbs and Fats. Which is C(10) + P(27.5) + F(10) = 12.5% <br>
** Activity Level ** <br>
I'll create this later, it will be a scale of activity/ stree that I create based on all the formulas/ studies above. 
I'll create functions for each one and then get the average of all the outcomes. <br>

# Harris Benedict Formula (BMR)

In [23]:
def HBbmr(height, weight, age, gender, weightMs, heightMs):
    
    if(heightMs == 'in'):
        height = height * 2.54
    if(weightMs == 'lbs'):
        weight = weight/2.20462
    
    if(gender == 'm'):
        bmr = 66.47+ (13.75 * weight) + ( 5.003 * height) - ( 6.755 * age)
    else:
        bmr = 655.1 + (9.563 * weight) + (1.850 * height) - (4.676 * age)
    
    return round(bmr)

# Revised Harris Benedict Formula (BMR)

In [24]:
def RHBbmr(height, weight, age, gender, weightMs, heightMs):
    
    if(heightMs == 'in'):
        height = height * 2.54
    if(weightMs == 'lbs'):
        weight = weight/2.20462
    
    if(gender == 'm'):
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    else:
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
        
    return round(bmr)

# The Mifflin St JEOR equation 

In [25]:
def MSJbmr(height, weight, age, gender, weightMs, heightMs):
    
    if(heightMs == 'in'):
        height = height * 2.54
    if(weightMs == 'lbs'):
        weight = weight/2.20462
    
    if(gender == 'm'):
        bmr =  9.99 * weight + 6.25 * height - 4.92 * age + 5; 
    else:
        bmr = 9.99 * weight + 6.25 * height - 4.92 * age - 161.
        
    return round(bmr)

# The Schofield Equation 

In [26]:
# The Schofield equation gives a 'standard error of estimated' or SEE to be added or subtraction based on body comp and age 
# It only gives to options either alot of lean bdy mass or not 
# I'm going to solit this up a bit by offering a low, medium, high, very high option for lean body mass for each age 

def Sbmr(weight, age, gender, weightMs, muscle):
    
    # metric check
    if(weightMs == 'lbs'):
        weight = weight/2.20462
    
    # Standard Error estimated/ SEE and BMR calculation
    
    if(gender == 'm'):        
        if(age >= 10 and age <= 17):
            bmr = 17.686 * (weight) + 658.2 
            see = 105
        if(age >= 18 and age <= 29):
            bmr = 15.057 * (weight) + 692.2 
            see = 153
        if(age >= 30 and age <= 59):
            bmr = 11.472 * (weight) + 873.1 
            see = 167
        if(age >= 60):
            bmr =11.711 * (weight)  + 587.7
            see = 164

    if(gender == 'f'):        
        if(age >= 10 and age <= 17):
            bmr = 13.384 * (weight)  + 692.6 
            see = 111
        if(age >= 18 and age <= 29):
            bmr = 14.818 * (weight)  + 486.6 
            see = 119
        if(age >= 30 and age <= 59):
            bmr = 8.126 * (weight)  + 845.6
            see = 111
        if(age >= 60):
            bmr = 9.082 * (weight)  + 658.5
            see = 108
        
    # Body compostion
    if(muscle == 'l'):
        see = see - see*2
    if(muscle == 'm'):
        see = see - see*1.5
    if(muscle == 'h'):
        see = see*.5
        
    
    return round(bmr + see)

# My BMR Formula (Average of four)

In [27]:
def BMRCAL(height, weight, age, gender, weightMs, heightMs, muscle):
    
    # BMR calculation 
    hbf = HBbmr(height, weight, age, gender, heightMs, weightMs)
    rhbf = RHBbmr(height, weight, age, gender, heightMs, weightMs)
    msjf = MSJbmr(height, weight, age, gender, heightMs, weightMs)
    sf = Sbmr(weight, age, gender, weightMs, muscle)
    
    return round((hbf + rhbf + msjf + sf)/ 4)

# Physical Actvity Scale (PA Scale)
The activity Scale is defined under categories. 
<br></br>
Sedentary.  Little to no regular exercise.  
(factor 1.0 - 1.370)
<br></br>
Mild activity level: Intensive exercise for at least 20 minutes 1 to 3 times per week. This may include such things as bicycling, jogging, basketball, swimming, skating, etc.  If you do not exercise regularly, but you maintain a busy life style that requires you to walk frequently for long periods, you meet the requirements of this level.  
(factor 1.375 - 1.50)
<br></br>
Moderate activity level: Intensive exercise for at least 30 to 60 minutes 3 to 4 times per week. Any of the activities listed above will qualify.    
(factor 1.55 - 1.65)
<br></br>
Heavy or (Labor-intensive) activity level: Intensive exercise for 60 minutes or greater 5 to 7 days per week (see sample activities above).  Labor-intensive occupations also qualify for this level.  Labor-intensive occupations include construction work (brick laying, carpentry, general labor, etc.). Also farming, landscape worker or similar occupations.     
(factor 1.7 - 1.85)
<br></br>
Extreme level: Exceedingly active and/or very demanding activities:  Examples include:  (1) athlete with an almost unstoppable training schedule with multiple training sessions throughout the day  (2) very demanding job, such as shoveling coal or working long hours on an assembly line. Generally, this level of activity is very difficult to achieve.
<br></br>
(factor 1.9 - 2.5)

# The EER Calulator 
The EER will take into consideration activity level/ Physical activity (PA) and TEF or Thermic effect of food which we defined above as 10%. Each function will apply the Scale factor given by the User differently and also the TEF <br>
The PA scale defined above will explain to the user which scale of PA they should use. <br> 
I'll use the average of three created function formulas to calculate EER. <br></br>
<br></br>
The first function will just take the PA scale factor given by the user and apllied the same way as the Harris Benedict and Mifflin formulas above did. <br>
(bmr x scale factor)<br>
The second function will take the PA and apply it the same way the Schofield formula does. <br>
(bmr x scale factor)*(gender)<br>
The third function will use the Institute of Medicines formulas combined with the PA given by the user.<br>
(newFormula x scale factor)<br>

In [28]:
def HBMeer(bmr, scaleF):
    return round((bmr*scaleF)*.90)

# Scohfield EER 

In [29]:
def Seer(bmr, scaleF, gender):
    
    
    if(gender == 'f'):
        if(scaleF >= 1.0 and scaleF <= 1.5):
            scaleF = scaleF*.9375
        if(scaleF > 1.5 and scaleF <= 1.65):
            scaleF = scaleF*.94
        if(scaleF > 1.65 and scaleF <= 1.85):
            scaleF = scaleF*.90
        if(scaleF > 1.85 and scaleF <= 2.5):
            scaleF = scaleF*.91
    
    return round((bmr*scaleF)*.90)

# Institute of Medicines 

In [30]:
def IOMeer(height, weight, age, gender, weightMs, heightMs, scaleF):
    
    if(heightMs == 'in'):
        height = height * 2.54
    if(weightMs == 'lbs'):
        weight = weight/2.20462
    if(heightMs == 'cm'):
        height = height/100
    
    
    if(gender == 'm'):
        if(age >= 3 and age <= 8):
            eer = 88.5 - (61.9 * age) + scaleF * (26.7 * weight) + (903 * height) +20
        if(age >= 9 and age <= 18):
            eer = 88.5 - (61.9 * age) + scaleF * (26.7 * weight) + (903 * height) + 25
        if(age >= 19):
            eer = 662 - (9.53 * age ) + scaleF *  (15.91 * weight) + (539.6 * height)
                                                  
    if(gender == 'f'):
        if(age >= 3 and age <= 8):
            eer = 135.3 - (30.8 * age) + scaleF * (10.0 * weight ) + (934 * height ) + 20
        if(age >= 9 and age <= 18):
             eer = 135.3 - (30.8 * age) + scaleF * (10.0 * weight) + (934 * height) + 25
        if(age >= 19):
            eer = 354 - (6.91 * age) + scaleF * (9.36 * weight) + (726 * height)
            
    return round(eer*.90)
        
    
    
    

# My EER Calculator (Average of four)

In [31]:
def eerCal(height, weight, age, gender, weightMs, heightMs, muscle, scaleF):

    hb = HBMeer(BMRCAL(height, weight, age, gender, weightMs, heightMs, muscle), scaleF)
    s = Seer(BMRCAL(height, weight, age, gender, weightMs, heightMs, muscle), scaleF, gender)
    imo = IOMeer(height, weight, age, gender, weightMs, heightMs, scaleF)
    
        
    return round((hb + s + imo)/3)

In [32]:
# BMR Functions Test
print("1: ", HBbmr(173, 80, 21, 'm', 'kg', 'cm'))
print("2: ", RHBbmr(173, 80, 21, 'm', 'kg', 'cm'))
print("3: ", MSJbmr(173, 80, 21, 'm', 'kg', 'cm'))
print("4: ", Sbmr(80, 21, 'm', 'kg', 'vh'))
print("5: ", BMRCAL(173, 80, 21, 'm', 'kg', 'cm', 'vh'))

1:  1890
2:  1871
3:  1782
4:  2050
5:  1898


In [33]:
# EER functions test 
print('1: ', HBMeer(BMRCAL(173, 80, 21, 'm', 'kg', 'cm', 'vh'), 2.0))
print('2: ', Seer(BMRCAL(173, 80, 21, 'm', 'kg', 'cm', 'vh'), 2.0, 'm'))
print('3: ', IOMeer(173, 80, 21, 'm', 'kg', 'cm', 2.0))
print('4: ', eerCal(173, 80, 21, 'm', 'kg', 'cm', 'vh', 2.0))

1:  3416
2:  3416
3:  3547
4:  3460


# BMR Calculation 
The BMR calculation will need the: <br>
Height - (cm/in) <br> 
Weight - (kg/lbs) <br> 
Age - (years) <br>
Gender - (m/f) <br>
Weight - (kg/lbs) in string form <br> 
Muscle - (l,m,h,vh) <br>
The user will enter this info through the use of a drop down menu and and mannual entry. <br>

In [117]:
# .E.G.
BMRCAL(173, 80, 21, 'm', 'kg', 'cm', 'vh')

1898

# EER Calculation 
The EER calculation will need the: <br>
Height - (cm/in) <br> 
Weight - (kg/lbs) <br> 
Age - (years) <br>
Gender - (m/f) <br>
Weight - (kg/lbs) in string form <br> 
Muscle - (l,m,h,vh) <br>
Scale Factor (choosen by the user as defined above) <br>
The user will enter this info through the use of a drop down menu and and mannual entry. <br>

In [118]:
# .E.G.
eerCal(173, 83, 50, 'm', 'kg', 'cm', 'l', 1.5)

2462

# BMI Calculator 
BMI is a person's weight in kilograms (kg) divided by his or her height in meters squared. The National Institutes of Health (NIH) now defines normal weight, overweight, and obesity according to BMI rather than the traditional height/weight charts. Overweight is a BMI of 27.3 or more for women and 27.8 or more for men.

In [191]:
def bmiCal(height, weight, heightMs, weightMs):
    
    if(heightMs == 'in'):
        height = height * 2.54
    if(weightMs == 'lbs'):
        weight = weight/2.20462
    
    bmi = round(weight/((height/100)**2), 1)
    bmiL = []
    bmiL.append(bmi)
    
    
    if (bmi <= 16):
        bmiL.append('Severe Thinness')
        print(bmiL[1], ": Consult Doctor/Nitritionist, consider gaining weight")
    if (bmi >= 16 and bmi <= 17):
        bmiL.append('Moderate Thinness')
        print(bmiL[1], ": Consider gaining weight")
    if (bmi >= 17 and bmi <= 18.5):
        bmiL.append('Mild Thinness')
        print(bmiL[1], ': Keep an Eye on your weight')
    if (bmi >= 18.5 and bmi <= 25):
        bmiL.append('Normal')
        print(bmiL[1], ": Within the nomral weight range")
    if (bmi >= 25 and bmi <= 30):
        bmiL.append('Overweight')
        print(bmiL[1], 'Consider losing weight')
    if (bmi >= 30 and bmi <= 35):
        bmiL.append('Obese Class 1')
        print(bmiL[1], 'Strongly advised losing weight')
    if (bmi >= 35 and bmi <= 40):
        bmiL.append('Obese Class 2')
        print(bmiL[1], 'Consult Doctor/ Nutritionist weight loss needed')
    if (bmi >= 40):
        bmiL.append('Obese Class 3')
        print(bmiL[1], 'Life shortening obesity weight loss crucial')
    
    return bmiL

In [192]:
bmiCal(173, 80, 'cm', 'kg')

Overweight Consider losing weight


[26.7, 'Overweight']

# Creating a User Profile 

In [208]:
class User:
    
    # in order for a user to create a profile we'll need this info off them 
    def __init__(self, name, ID, height, weight, age, gender, weightMs, heightMs):
        # Basic varaibles 
        self.name = name
        self.userID = ID
        self.height = height 
        self.weight = weight 
        self.age  = age 
        self.gender = gender 
        self.weightMs = weightMs
        self.heightMs = heightMs
        self.foodTDays = {}
        
        # add a day of eating 
    def addFoodDay(self, date, dayM):
         self.foodTDays[date] = dayM

        # get users BMR based on stats 
    def getBMR(self, muscle):
        self.BMR = BMRCAL(self.height, self.weight, self.age, self.gender, self.weightMs, self.heightMs, muscle)
        self.muscle = muscle 
        return self.BMR
    
        # get users eer based on stats 
    def getEER(self, scaleF):
        self.EER = eerCal(self.height, self.weight, self.age, self.gender, self.weightMs, self.heightMs, self.muscle, scaleF)
        self.scaleF = scaleF
        return self.EER
    
    def getBMI(self):
        self.BMIL = bmiCal(self.height, self.weight, self.heightMs, self.weightMs)
        self.BMI = self.BMIL[0]
        return self.BMIL
    
    # setting users goals to generate macros and in the future weight gain and loss plans 
    def setGoal(self, goal):
        self.goal = goal 
    
    # displaying user stats     
    def displayStats(self):
        StatDF = pd.DataFrame(columns=['Name', 'Age', 'User ID', 'Height', 'Weight'])
        StatDF.loc[0] = np.array([self.name, self.age, self.userID, str(self.height) + self.heightMs, str(self.weight) + self.weightMs])
        StatDF = StatDF.set_index('Name').T
        self.statDF = StatDF
        return self.statDF
    
    # displaying the days a user has track their food intake 
    def displayDaysTracked(self):
        return self.foodTDays.keys()
        
    # returning the dictionary containing food days tracked and coresponding dates 
    def displayFoodTrackDict(self):
        return self.foodTDays
        

# Sample User

In [209]:
# Entering Stats
user1 = User('Dan Lenehan', 100, 173, 80, 21, 'm', 'kg', 'cm')

In [210]:
# Displaying stats
user1.displayStats()

Name,Dan Lenehan
Age,21
User ID,100
Height,173cm
Weight,80kg


In [211]:
# Getting calories entering the users muscle description and Scale factor of daily actvity  
print("BMR: ", user1.getBMR('vh'))
print("EER: ", user1.getEER(2))
print("BMI: ", user1.getBMI())

BMR:  1898
EER:  3460
Overweight Consider losing weight
BMI:  [26.7, 'Overweight']


In [212]:
# Adding day of eating created in the last notebook 
user1.addFoodDay(dayTemp.displayDate(), dayTemp)

In [198]:
# Days Tracked
user1.displayDaysTracked()

dict_keys([datetime.date(2018, 6, 19)])

In [199]:
# The dictionary 
print(user1.displayFoodTrackDict())

{datetime.date(2018, 6, 19): <__main__.dayM object at 0x000001701EF68908>}


# Macro Nutrient Calculation
As far as macro nutrients go we have the calories so now we can get macro nutrients to fit these calories. The easiest and best way in my opinion to do this is to get the minium/ most optimal amount of fat and protein calories to achieve certain goals. After we have that the rest of the calories can be from carbs.